In [37]:
%pip install langchain
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [38]:
import os
import openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain import OpenAI, ConversationChain
from langchain.chains import SequentialChain

In [39]:
from dotenv import load_dotenv

openai.api_key = os.environ['API_KEY']
os.environ["OPENAI_API_KEY"] = os.environ['API_KEY']

In [40]:
mytalk = """Aさん：今度の新年会はいつにしますか。
Bさん：私は１月下旬がちょうどいいです。
Cさん：私は1月だと1/20か、1/30がちょうどいいです。
Dさん：私は1月だと参加は難しそうです。
Aさん：じゃあいったん1/30にしましょうか。
Bさん：わかりました。時間は何時にします？
Aさん：そうですね、じゃあ18:00にしましょうか。
Cさん：ちょっと早くないですか？19:00はどうでしょう。
Aさん：了解です。じゃあ19時で！場所はいつもの黒木屋にしましょうか。
Cさん：了解です。楽しみにしてます。
Dさん：楽しんできてください。いけそうだったらまた連絡します。
"""

In [41]:
event_prompt = PromptTemplate(
      input_variables=["talk"],
      template="""次の文章の中から開催されるイベント名を"イベント名：？？？"の形で返してください。不明な場合は"イベント名：未定"を返してください。
      {talk}
      """)

event_chain = LLMChain(
      llm=llm, 
      prompt=event_prompt,
      output_key='event')
  

date_prompt = PromptTemplate(
      input_variables=["talk"],
      template="""次の文章の中から開催されるイベントの開催日を"開催日：？？？"の形で返してください。不明な場合は"開催日：未定"を返してください。
      {talk}
      """)
  
date_chain = LLMChain(
      llm=llm, 
      prompt=date_prompt,
      output_key='date')


time_prompt = PromptTemplate(
      input_variables=["talk"],
      template="""次の文章の中から開催されるイベントの開始時間を"開始時間：？？？"の形で返してください。不明な場合は"開催時間：未定"を返してください。
      {talk}
      """)

time_chain = LLMChain(
      llm=llm, 
      prompt=time_prompt,
      output_key='time')


place_prompt = PromptTemplate(
      input_variables=["talk"],
      template="""次の文章の中から開催されるイベントの開催場所を"開催場所：？？？"の形で返してください。不明な場合は"開催場所：未定"を返してください。
      {talk}
      """)


place_chain = LLMChain(
      llm=llm, 
      prompt=place_prompt,
      output_key='place')


sanka_prompt = PromptTemplate(
      input_variables=["talk"],
      template="""次の文章の中から開催されるイベントの参加予定者を"参加予定者：？？？"の形で返してください。不明な場合は"参加予定者：未定"を返してください。
      {talk}
      """)

sanka_chain = LLMChain(
      llm=llm, 
      prompt=sanka_prompt,
      output_key='sanka')


In [42]:
overall_chain = SequentialChain(
    chains = [event_chain, date_chain, time_chain, place_chain, sanka_chain],
    input_variables = ["talk"],
    output_variables = ["event", "date", "time", "place", "sanka"],
    verbose = True
)

In [43]:
event_info = overall_chain({
    'talk':mytalk
})



> Entering new SequentialChain chain...

> Finished chain.


In [44]:
event_info

{'talk': 'Aさん：今度の新年会はいつにしますか。\nBさん：私は１月下旬がちょうどいいです。\nCさん：私は1月だと1/20か、1/30がちょうどいいです。\nDさん：私は1月だと参加は難しそうです。\nAさん：じゃあいったん1/30にしましょうか。\nBさん：わかりました。時間は何時にします？\nAさん：そうですね、じゃあ18:00にしましょうか。\nCさん：ちょっと早くないですか？19:00はどうでしょう。\nAさん：了解です。じゃあ19時で！場所はいつもの黒木屋にしましょうか。\nCさん：了解です。楽しみにしてます。\nDさん：楽しんできてください。いけそうだったらまた連絡します。\n',
 'event': ' イベント名：新年会',
 'date': ' 開催日：1/30',
 'time': ' 開始時間：19:00',
 'place': ' 開催場所：黒木屋',
 'sanka': ' 参加予定者：Aさん、Bさん、Cさん'}

In [45]:
eventlist = event_info['event']+event_info['date']+event_info['time']+event_info['place']+event_info['sanka']

In [46]:
eventsummary_prompt = PromptTemplate(
    input_variables=["talk"],
    template="""次のリスト情報をもとに開催されるイベントの概要を返してください。
    
    {talk}
    """
)

eventsummary_chain = LLMChain(
    llm=llm, 
    prompt=eventsummary_prompt)
    

summary_ans = eventsummary_chain.run({
    'talk':eventlist
})

In [47]:
print(summary_ans)



1月30日（水）19:00より、黒木屋にて新年会を開催いたします。Aさん、Bさん、Cさんの参加予定です。ご参加をお待ちしております。
